Part 1

In [ ]:
openai_api_key = ""

In [ ]:
pip install requests


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import requests

def get_top_places_from_yelp(location):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "Authorization": ""
    }
    params = {
        "term": "tourist attractions",
        "location": location,
        "limit": 10
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    places = []
    for business in data['businesses']:
        name = business['name']
        address = ", ".join(business['location']['display_address'])
        places.append(f"{name} ({address})")

    return places


In [ ]:
import openai

def generate_plan_with_chat_gpt(places, budget, days):
    openai.api_key = ""

    places_text = "\n".join(places)
    prompt = f"Create a detailed travel itinerary for {days} days within a budget of {budget} including the following places:\n{places_text}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",   # Use the ChatGPT model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    generated_text = response.choices[0].text
    return generated_text



In [ ]:
def main():
    location = "San Francisco, CA"
    budget = "$1000"
    days = "3"

    print("Fetching top places to visit...")
    top_places = get_top_places_from_yelp(location)
    for idx, place in enumerate(top_places, start=1):
        print(f"{idx}. {place}")

    print("\nGenerating travel plan...")
    plan = generate_plan_with_chat_gpt(top_places, budget, days)
    print(plan)

if __name__ == "__main__":
    main()


In [ ]:
import openai

def generate_plan(location, budget, days):
    """
    Use ChatGPT API Generate a detailed plan based on location, budget, and days.
    :param location: locations
    :param budget: budgets
    :param days: days
    :return: Plan for model generation
    """

    # Set your OpenAI API key
    openai.api_key = ''

    # Build prompt
    prompt = (
        f"Create a detailed travel plan with activities, meals and accommodation suggestions。\n"
        f"location：{location}\n"
        f"budget：{budget}\n"
        f"days:{days}\n"
        f"Be sure to provide a daily itinerary with interesting activities, recommended restaurants and suitable places to stay.\n"
    )

    # Invoke the OpenAI ChatGPT API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",   # Use the ChatGPT model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # Gets the generated text
    generated_text = response['choices'][0]['message']['content']

    return generated_text

#input
location = "Tianjin, China"
budget = "1000 CNY"
days = "5 days"

# Call the function and print the result
plan = generate_plan(location, budget, days)
print(plan)

Day 1:
- Morning: Start your trip by exploring the Ancient Cultural Street. Experience the traditional Chinese culture as you stroll along this vibrant street filled with shops, restaurants, and street performers.
- Lunch: Enjoy local cuisine at Guifaxiang Pastry, known for its delicious traditional pastries and snacks.
- Afternoon: Visit the Tianjin Eye, an iconic Ferris wheel that offers panoramic views of the city. Take a ride for a unique perspective of Tianjin.
- Dinner: Head to Goubuli Steamed Buns, a famous restaurant known for its delicious baozi (steamed buns). Try the Goubuli Baozi, a must-try specialty.
- Accommodation: Stay at the Astor Hotel Tianjin, a historic hotel with a great location in the heart of the city.

Day 2:
- Morning: Visit the Porcelain House, a unique attraction that showcases an impressive collection of Chinese porcelain artworks incorporated into the architecture.
- Lunch: Try some local seafood at Xiaobailou Seafood Street, where you can find a variety 

Part 2

In [ ]:
import re

def extract_locations(text):
    # A simple regex to find capitalized words
    locations = re.findall(r'\b[A-Z][a-z]*\b', text)
    return list(set(locations))  # Removing duplicates

In [ ]:
pip install spacy

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def extract_locations(text):
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ in ['GPE', 'LOC']]
    return list(set(locations))  # Removing duplicates

In [ ]:
locations = extract_locations(plan)
print(locations)

['Tianjin', 'the Haihe River', 'Huangyaguan', 'Taipingzhai']


Part 3

In [ ]:
!pip install requests folium
import requests
import folium
from folium.plugins import MarkerCluster

In [ ]:
# exmaples(just for test)
location = "San Francisco, CA"
locations = [
    "The 16th Avenue Tiled Steps (Moraga St. between 15th & 16th Ave, San Francisco, CA 94122)",
    "Umbrella Alley San Francisco (757 Beach St, San Francisco, CA 94108)",
    "Fort Point (San Francisco, CA 94129)",
    "Twin Peaks (501 Twin Peaks Blvd, San Francisco, CA 94114)",
    "Hidden Garden Steps (16th Ave at Kirkham St, San Francisco, CA 94122)",
    "Entwined Lighted Forest (240 John F Kennedy Dr, San Francisco, CA 94117)",
    "Japanese Tea Garden (75 Hagiwara Tea Garden Dr, Golden Gate Park, San Francisco, CA 94118)",
    "Painted Ladies (700 Steiner St, San Francisco, CA 94117)",
    "The Glass Elevators (335 Powell St, Westin St. Francis Hotel, San Francisco, CA 94102)",
    "Lombard Street (1000-1099 Lombard St, San Francisco, CA 94133)"
]
transpotation_type = "driving"

In [ ]:
import os
import requests
import folium
from folium.plugins import MarkerCluster

API_KEY = ''

# Get coordinates for a specific location using Google Maps Geocoding API
def get_coordinates(location):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

for loc in locations:
    coord = get_coordinates(loc)
    if coord:
        print(f"{loc}: {coord}")

The 16th Avenue Tiled Steps (Moraga St. between 15th & 16th Ave, San Francisco, CA 94122): (37.7562439, -122.4732299)
Umbrella Alley San Francisco (757 Beach St, San Francisco, CA 94108): (37.8065132, -122.4212258)
Fort Point (San Francisco, CA 94129): (37.8106117, -122.47709)
Twin Peaks (501 Twin Peaks Blvd, San Francisco, CA 94114): (37.7529149, -122.4474131)
Hidden Garden Steps (16th Ave at Kirkham St, San Francisco, CA 94122): (37.7598172, -122.4735837)
Entwined Lighted Forest (240 John F Kennedy Dr, San Francisco, CA 94117): (37.771614, -122.457647)
Japanese Tea Garden (75 Hagiwara Tea Garden Dr, Golden Gate Park, San Francisco, CA 94118): (37.7700913, -122.4704359)
Painted Ladies (700 Steiner St, San Francisco, CA 94117): (37.7762528, -122.4327556)
The Glass Elevators (335 Powell St, Westin St. Francis Hotel, San Francisco, CA 94102): (37.7876827, -122.4087648)
Lombard Street (1000-1099 Lombard St, San Francisco, CA 94133): (37.802139, -122.41874)


In [ ]:
import requests
import folium
from folium.plugins import MarkerCluster

def get_coordinates(location):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        latlng = data['results'][0]['geometry']['location']
        return latlng['lat'], latlng['lng']
    else:
        print(f"Error fetching coordinates for {location}. Error: {data['status']}")
        return None

def tsp_greedy(matrix):
    num_locations = len(matrix)
    unvisited = set(range(num_locations))
    path = [0]
    unvisited.remove(0)

    while unvisited:
        last = path[-1]
        next_loc = min(unvisited, key=lambda x: matrix[last][x])
        path.append(next_loc)
        unvisited.remove(next_loc)
    path.append(path[0])
    return path

def get_chunked_distance_matrix(locations):
    max_elements = 10  # Google's limitation is typically 10 for free tier
    num_locations = len(locations)
    matrix = [[0 for _ in range(num_locations)] for _ in range(num_locations)]

    for i in range(0, num_locations, max_elements):
        for j in range(0, num_locations, max_elements):
            chunked_origins = locations[i:i+max_elements]
            chunked_destinations = locations[j:j+max_elements]
            chunk_matrix = get_distance_matrix(chunked_origins, chunked_destinations)

            if chunk_matrix is None:
                print("Error fetching a chunk of distance matrix.")
                return None

            for x, row in enumerate(chunk_matrix):
                for y, distance in enumerate(row):
                    matrix[i+x][j+y] = distance
    return matrix

def get_distance_matrix(origins, destinations=None):
    if destinations is None:
        destinations = origins

    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
    params = {
        "origins": "|".join(origins),
        "destinations": "|".join(destinations),
        "mode": transpotation_type,
        "key": API_KEY
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        matrix = [[row['distance']['value'] for row in entry['elements']] for entry in data['rows']]
        return matrix
    else:
        print(f"Error fetching distance matrix. Error: {data['status']}")
        return None

def plot_with_marker_cluster(locations, tuple_coordinates, base_location):
    base_coord = get_coordinates(base_location)
    m = folium.Map(location=base_coord, zoom_start=13)
    marker_cluster = MarkerCluster().add_to(m)
    for i, (location, coord) in enumerate(zip(locations, tuple_coordinates)):
        if coord:
            folium.Marker(
                location=coord,
                popup=location,
                icon=folium.DivIcon(html=f'<div style="font-size: 12pt; color : black">{i+1}</div>'),
            ).add_to(marker_cluster)

    # Draw polyline without the starting and ending location
    folium.PolyLine(tuple_coordinates, color="blue", weight=2.5, opacity=1).add_to(m)

    return m

full_locations = [location] + locations

coordinates = [get_coordinates(loc) for loc in [location] + locations]

# Using the chunked version to get distance matrix
distance_matrix = get_chunked_distance_matrix(full_locations)
if distance_matrix is None:
    print("Error getting distance matrix. Execution stopped.")
else:
    try:
        optimized_order = tsp_greedy(distance_matrix)
        optimized_coords = [coordinates[i] for i in optimized_order]
        optimized_locations = [full_locations[i] for i in optimized_order]

        # Remove the first and last elements which represent the starting and ending location
        optimized_coords = optimized_coords[1:-1]
        optimized_locations = optimized_locations[1:-1]

        map_object = plot_with_marker_cluster(optimized_locations, optimized_coords, location)
        display(map_object)
    except KeyError as e:
        print(f"An error occurred: {e}")